In [1]:
#make matrix for deseq
import sys
sys.path.insert(0, "/home/gstupp/projects/metaproteomics")

In [8]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter
from glob import glob
import numpy as np
from itertools import chain
import pandas as pd
from collections import ChainMap
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [9]:
from metaproteomics.file_processing.blazmass_tools import dta_select_parser, build_pep_quant_dict

In [10]:
os.getcwd()

'/home/gstupp/projects/Wolan/ana_extractions/analysis/analysis_deseq'

In [11]:
files = glob("data/*/*/*.txt")
files[:2]

['data/PA/Pellet/PA pellet 2.txt', 'data/PA/Pellet/PA pellet 3.txt']

In [12]:
meta = pd.DataFrame([{"cell": s.rsplit("/", 1)[-1].split(" ")[0], 
  "method": s.rsplit("/", 1)[-1].split(" ")[1],
  "name": s.rsplit("/", 1)[-1].split(".")[0].replace(" ", "_"),
  "pretty_name": s.rsplit("/", 1)[-1].split(".")[0],
  "file": s } for s in files])
meta.index = meta.name
meta.to_csv("metadata.csv")
meta

,cell,file,method,name,pretty_name
name,,,,,
PA_pellet_2,PA,data/PA/Pellet/PA pellet 2.txt,pellet,PA_pellet_2,PA pellet 2
PA_pellet_3,PA,data/PA/Pellet/PA pellet 3.txt,pellet,PA_pellet_3,PA pellet 3
PA_pellet_1,PA,data/PA/Pellet/PA pellet 1.txt,pellet,PA_pellet_1,PA pellet 1
PA_triflic_2,PA,data/PA/Triflic/PA triflic 2.txt,triflic,PA_triflic_2,PA triflic 2
PA_triflic_3,PA,data/PA/Triflic/PA triflic 3.txt,triflic,PA_triflic_3,PA triflic 3
PA_triflic_1,PA,data/PA/Triflic/PA triflic 1.txt,triflic,PA_triflic_1,PA triflic 1
PA_sonic_1,PA,data/PA/Sonic/PA sonic 1.txt,sonic,PA_sonic_1,PA sonic 1
PA_sonic_3,PA,data/PA/Sonic/PA sonic 3.txt,sonic,PA_sonic_3,PA sonic 3
PA_sonic_2,PA,data/PA/Sonic/PA sonic 2.txt,sonic,PA_sonic_2,PA sonic 2


In [13]:
def format_id(name):
    return name.split("|")[1] if "|" in name else name

In [14]:
def pick_descriptive_name(locus):
    return locus['loci'][0]['Descriptive Name']

In [15]:
uniprot_name = [{format_id(x['name']): pick_descriptive_name(x) for x in dta_select_parser(file, return_reverse=False)} for file in files]
uniprot_name = { k: v for d in uniprot_name for k, v in d.items() }

In [16]:
def get_locus_quantification(locus):
    return {format_id(x['description']):x['Spectrum Count'] for x in locus['loci']}
def get_file_quantifications(file):
    return dict(ChainMap(*[get_locus_quantification(x) for x in dta_select_parser(file, return_reverse=False)]))
def get_file_name(file):
    return file.rsplit("/", 1)[-1].split(".")[0].replace(" ", "_")

In [17]:
df = pd.DataFrame({get_file_name(file): get_file_quantifications(file) for file in files})
df.head()

,BS_pellet_1,BS_pellet_2,BS_pellet_3,BS_sonic_1,BS_sonic_2,BS_sonic_3,BS_triflic_1,BS_triflic_2,BS_triflic_3,Jurkat_sonic_1,...,Jurkat_triflic_3,PA_pellet_1,PA_pellet_2,PA_pellet_3,PA_sonic_1,PA_sonic_2,PA_sonic_3,PA_triflic_1,PA_triflic_2,PA_triflic_3
A0A024QZP7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0A024R216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0A024R341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0A024R4M0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0A024R571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.loc['Q7L7L0']

BS_pellet_1           NaN
BS_pellet_2           NaN
BS_pellet_3           NaN
BS_sonic_1            NaN
BS_sonic_2            NaN
BS_sonic_3            NaN
BS_triflic_1          NaN
BS_triflic_2          NaN
BS_triflic_3          NaN
Jurkat_sonic_1      175.0
Jurkat_sonic_2      100.0
Jurkat_sonic_3      253.0
Jurkat_triflic_1    111.0
Jurkat_triflic_2     62.0
Jurkat_triflic_3    156.0
PA_pellet_1           NaN
PA_pellet_2           NaN
PA_pellet_3           NaN
PA_sonic_1            NaN
PA_sonic_2            NaN
PA_sonic_3            NaN
PA_triflic_1          NaN
PA_triflic_2          NaN
PA_triflic_3          NaN
Name: Q7L7L0, dtype: float64

In [19]:
df.to_csv("X.csv")

In [11]:
# run deseq.R with X.csv

In [12]:
### Add a name column and create ranked gene lists for gsea

In [61]:
#*** BS ***
bs = pd.read_csv("BS_deseq_results.csv", index_col=0)
bs['name'] = bs.index.map(uniprot_name.get)
bs.to_csv("BS_deseq_results.csv")
bs.head(2)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,name
P80239,747.568119,-4.366607,0.299565,-14.576497,3.963404e-48,4.383525e-45,Alkyl hydroperoxide reductase subunit C OS=Bac...
P54375,329.408983,5.712140,0.437433,13.058305,5.698257e-39,3.151136e-36,Superoxide dismutase [Mn] OS=Bacillus subtilis...


In [62]:
# write ranked list for gsea
(-1*np.log(bs.padj.dropna())).to_csv("bs.rnk", sep="\t")
# write filtered ranked list
(-1*np.log(bs[bs.padj<0.2].padj.dropna())).to_csv("bs_filtered.rnk", sep="\t")

In [63]:
#*** PA ***
pa = pd.read_csv("PA_deseq_results.csv", index_col=0)
pa['name'] = pa.index.map(uniprot_name.get)
pa.to_csv("PA_deseq_results.csv")
pa.head(2)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,name
Q9HWW1,556.096497,5.233479,0.501978,10.425719,1.892232e-25,2.210127e-22,Outer membrane protein OprG OS=Pseudomonas aer...
P13794,546.292125,7.079937,0.729996,9.698594,3.056804e-22,1.785173e-19,Outer membrane porin F OS=Pseudomonas aerugino...


In [64]:
(-1*np.log(pa.padj.dropna())).to_csv("pa.rnk", sep="\t")
(-1*np.log(pa[pa.padj<0.2].padj.dropna())).to_csv("pa_filtered.rnk", sep="\t")

In [65]:
#*** Jurkat ***
j = pd.read_csv("Jurkat_deseq_results.csv", index_col=0)
j['name'] = j.index.map(uniprot_name.get)
j.to_csv("Jurkat_deseq_results.csv")
j.head(2)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,name
H3BR70,226.095015,-3.799501,0.699269,-5.433531,5.524981e-08,0.000122,Pyruvate kinase OS=Homo sapiens GN=PKM PE=1 SV=1
B4DNK4,286.158424,-3.439163,0.691375,-4.974385,6.545502e-07,0.000721,Pyruvate kinase OS=Homo sapiens GN=PKM PE=1 SV=1


In [66]:
(-1*np.log(j.padj.dropna())).to_csv("j.rnk", sep="\t")
(-1*np.log(j[j.padj<0.2].padj.dropna())).to_csv("j_filtered.rnk", sep="\t")

In [67]:
##
# map uniprot to hgnc
def uniprot_to_hgnc(uniprot):
    try:
        url = "https://mygene.info/v3/query?q=uniprot.Swiss-Prot:{}&fields=HGNC,symbol".format(uniprot)
        return requests.get(url).json()['hits'][0]['symbol']
    except Exception:
        return ""
uniprot_to_hgnc("P42771")

'CDKN2A'

In [68]:
uniprot_hgnc = dict()
for x in tqdm(j.index):
    uniprot_hgnc[x] = uniprot_to_hgnc(x)

100%|██████████| 5272/5272 [18:25<00:00,  4.77it/s]


In [69]:
j['hgnc'] = j.index.map(uniprot_hgnc.get)

In [70]:
j_hgnc = j.copy()
j_hgnc.index = j.hgnc
(-1*np.log(j_hgnc.padj.dropna())).to_csv("j_hgnc.rnk", sep="\t")

In [71]:
j_hgnc

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,name,hgnc
hgnc,,,,,,,,
,226.095015,-3.799501,0.699269,-5.433531,5.524981e-08,0.000122,Pyruvate kinase OS=Homo sapiens GN=PKM PE=1 SV=1,
,286.158424,-3.439163,0.691375,-4.974385,6.545502e-07,0.000721,Pyruvate kinase OS=Homo sapiens GN=PKM PE=1 SV=1,
ATP5IF1,24.407240,-7.667422,1.590492,-4.820785,1.429945e-06,0.001050,"ATPase inhibitor, mitochondrial OS=Homo sapien...",ATP5IF1
,24.489906,5.146161,1.124702,4.575576,4.749116e-06,0.002616,Heterogeneous nuclear ribonucleoprotein M (Fra...,
,25.257971,4.262155,0.964187,4.420467,9.848793e-06,0.004339,Heterogeneous nuclear ribonucleoprotein M (Fra...,
,64.793259,-4.363920,0.996679,-4.378459,1.195215e-05,0.004388,Tubulin beta-4A chain (Fragment) OS=Homo sapie...,
,28.972920,4.212889,1.035837,4.067137,4.759427e-05,0.014470,Histone H2A OS=Homo sapiens GN=H2AFV PE=3 SV=1,
HIST1H3H,199.507236,-3.048879,0.753926,-4.044002,5.254648e-05,0.014470,Histone H3.1 OS=Homo sapiens GN=HIST1H3A PE=1 ...,HIST1H3H
PKM,323.412407,-2.617861,0.655070,-3.996309,6.433779e-05,0.015748,Pyruvate kinase PKM OS=Homo sapiens GN=PKM PE=...,PKM
